In [152]:
import pypsa
import matplotlib.pyplot as plt
import pandas as pd

# rootPath = "../../results/base27Node2045"
rootPath = "../../results/scenario/geothermal"


plt.style.use('bmh')
%matplotlib inline

n = pypsa.Network(rootPath + "/postnetworks/base_s_27_lvopt_3H_3H_2045.nc")

INFO:pypsa.io:Imported network base_s_27_lvopt_3H_3H_2045.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


In [15]:
def new_grouper(n, c):
  return n.df(c).index.to_series()

n.s

In [50]:
n.objective / 1e9

522.7325723980724

In [140]:
df = pd.read_csv(rootPath + "/csvs/costs.csv", skiprows=7, header=None)

n.objective

522732572398.07245

In [151]:
import pypsa

pd.set_option('display.width', 100000)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


def de_grouper(n, c):
  df = n.df(c)
  specific_indices = ['10', '13', '14', '16', '17', '18', '19', '23', '24', '25','26', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37','4', '5']
  indexs = df[df.index.astype(str).str.contains('DE0') | df.index.astype(str).isin(specific_indices)].index.to_series()
  # n.df(c).index.to_series()
  return indexs
pypsa.statistics.groupers.add_grouper("de_grouper", de_grouper)

def eu_grouper(n, c):
  df = n.df(c)
  indexs = df[df.index.astype(str).str.contains('EU')].index.to_series()
  return indexs
pypsa.statistics.groupers.add_grouper("eu_grouper", eu_grouper)


# print(n.statistics.capex(groupby="new_grouper").sum() / 1e9)

# print(n.statistics.opex(groupby="new_grouper").sum() / 1e9)

print(n.statistics.expanded_capex(groupby="de_grouper").sum() / 1e9)
# print(n.statistics.expanded_capex().sum() / 1e9)

n.statistics.transmission(groupby=["carrier","de_grouper"] ).sum()

36.61302250898738


99014124.26165

In [66]:
# 成本

df = pd.read_csv(rootPath + "/csvs/nodal_costs.csv", skiprows=7, header=None)

print('total EU:', df[4].sum()/1e9)

filtered_df = df[df[2].isin(['DE0 0', 'DE0 1', 'DE0 2', 'DE0 3', 'DE0 4', 'DE0 5', 'DE0 6', 'DE0 7'])]

print('total DE:', filtered_df[4].sum()/1e9)

pd.set_option('display.float_format', '{:.4f}'.format)
pd.set_option('display.max_rows', None)


filtered_df.groupby(3)[4].sum()/1e6 # M EUR

total EU: 670.0127581246344
total DE: 53.81659545627004


3
BEV charger                                  1.5999
DAC                                        132.7799
EV battery                                   0.0000
Fischer-Tropsch                            642.6110
H2 Electrolysis                           3312.8955
H2 Fuel Cell                                 0.0012
H2 Store                                   120.9603
OCGT                                         0.0015
PHS                                       1375.0512
SMR                                          0.0013
SMR CC                                       0.0016
Sabatier                                     0.0009
V2G                                          0.0518
battery                                      0.0021
battery charger                              0.0031
battery discharger                           0.0000
electricity distribution grid             5312.3306
geothermal district heat                     0.0011
geothermal heat                              0.0233
geothermal

In [53]:
# n.explore(components= "Bus", popup = False, tiles= "CartoDB Positron")

In [83]:
# energy supply

df = pd.read_csv(rootPath + "/csvs/nodal_supply_energy.csv", skiprows=4, header=None)


filtered_df = df #df[df[2].isin(['DE0 0', 'DE0 1', 'DE0 2', 'DE0 3', 'DE0 4', 'DE0 5', 'DE0 6', 'DE0 7'])]

print('total DE', filtered_df[4].sum()/1e9)

pd.set_option('display.float_format', '{:.4f}'.format)
pd.set_option('display.max_rows', None)

filtered_df.groupby(3)[4].sum() # MW

total DE 4.2224526405334474e-11


3
AC0                                         -62209981.6236
AC1                                          52630623.5986
BEV charger0                               -809919993.4905
BEV charger1                                728927994.1414
DAC0                                       -133863233.5915
DAC1                                       -340742776.4146
DAC2                                       -243387697.4390
DAC3                                        243387697.4390
DC0                                       -1133989678.2066
DC1                                        1087719940.4728
EV battery                                          0.0003
Fischer-Tropsch0                          -1626955632.2149
Fischer-Tropsch1                           1299937550.1397
Fischer-Tropsch2                           -334213944.1409
Fischer-Tropsch3                             61417560.3422
H2 Electrolysis0                          -3735262786.7872
H2 Electrolysis1                           2194093360.

In [104]:
def calculate_energy(n):

    for c in n.iterate_components(n.one_port_components | n.branch_components):
        if c.name in n.one_port_components:
            c_energies = (
                c.pnl.p.multiply(n.snapshot_weightings.generators, axis=0)
                .sum()
                .multiply(c.df.sign)
                .groupby(c.df.carrier)
                .sum()
            )
        else:
            c_energies = pd.Series(0.0, c.df.carrier.unique())
            for port in [col[3:] for col in c.df.columns if col[:3] == "bus"]:
                totals = (
                    c.pnl["p" + port]
                    .multiply(n.snapshot_weightings.generators, axis=0)
                    .sum()
                )
                # remove values where bus is missing (bug in nomopyomo)
                no_bus = c.df.index[c.df["bus" + port] == ""]
                totals.loc[no_bus] = float(
                    n.component_attrs[c.name].loc["p" + port, "default"]
                )
                c_energies -= totals.groupby(c.df.carrier).sum()

        c_energies = pd.concat([c_energies], keys=[c.list_name])
        return c_energies

energy = calculate_energy(n)

print(len(energy))

21


In [124]:
def calculate_supply_energy(n, label, supply_energy):
    """
    Calculate the total energy supply/consuption of each component at the buses
    aggregated by carrier.
    """
    bus_carriers = n.buses.carrier.unique()

    for i in bus_carriers:
        bus_map = n.buses.carrier == i
        bus_map.at[""] = False

        for c in n.iterate_components(n.one_port_components):
            items = c.df.index[c.df.bus.map(bus_map).fillna(False)]
            items = [element for element in items if 'DE0' in element]

            if len(items) == 0:
                continue

            s = (
                c.pnl.p[items]
                .multiply(n.snapshot_weightings.generators, axis=0)
                .sum()
                .multiply(c.df.loc[items, "sign"])
                .groupby(c.df.loc[items, "carrier"])
                .sum()
            )
            s = pd.concat([s], keys=[c.list_name])
            s = pd.concat([s], keys=[i])

            supply_energy = supply_energy.reindex(s.index.union(supply_energy.index))
            supply_energy.loc[s.index, label] = s

        for c in n.iterate_components(n.branch_components):
            for end in [col[3:] for col in c.df.columns if col[:3] == "bus"]:
                items = c.df.index[c.df[f"bus{str(end)}"].map(bus_map).fillna(False)]
                items = [element for element in items if 'DE0' in element]
                if len(items) == 0:
                    continue

                s = (-1) * c.pnl["p" + end][items].multiply(
                    n.snapshot_weightings.generators, axis=0
                ).sum().groupby(c.df.loc[items, "carrier"]).sum()
                s.index = s.index + end
                s = pd.concat([s], keys=[c.list_name])
                s = pd.concat([s], keys=[i])

                supply_energy = supply_energy.reindex(
                    s.index.union(supply_energy.index)
                )

                supply_energy.loc[s.index, label] = s

    return supply_energy

columns = pd.MultiIndex.from_tuples(
        [(27, 'vopt', '3H3H', 2045)],
        names=["cluster", "ll", "opt", "planning_horizon"],
    )

supply_energy = pd.DataFrame(columns=columns, dtype=float)

# calculate_supply_energy(n, (27, 'vopt', '3H3H', 2045), supply_energy)



In [131]:
n.controllable_one_port_components 

{'Generator', 'Load', 'StorageUnit', 'Store'}

In [ ]:
opt_name = {"Store": "e", "Line": "s", "Transformer": "s"}

def calculate_costs(n, label, costs):
    for c in n.iterate_components(
        n.branch_components | n.controllable_one_port_components ^ {"Load"}
    ):
        capital_costs = c.df.capital_cost * c.df[opt_name.get(c.name, "p") + "_nom_opt"]
        capital_costs_grouped = capital_costs.groupby(c.df.carrier).sum()

        capital_costs_grouped = pd.concat([capital_costs_grouped], keys=["capital"])
        capital_costs_grouped = pd.concat([capital_costs_grouped], keys=[c.list_name])

        costs = costs.reindex(capital_costs_grouped.index.union(costs.index))

        costs.loc[capital_costs_grouped.index, label] = capital_costs_grouped

        if c.name == "Link":
            p = c.pnl.p0.multiply(n.snapshot_weightings.generators, axis=0).sum()
        elif c.name == "Line":
            continue
        elif c.name == "StorageUnit":
            p_all = c.pnl.p.multiply(n.snapshot_weightings.generators, axis=0)
            p_all[p_all < 0.0] = 0.0
            p = p_all.sum()
        else:
            p = c.pnl.p.multiply(n.snapshot_weightings.generators, axis=0).sum()

        # correct sequestration cost
        if c.name == "Store":
            items = c.df.index[
                (c.df.carrier == "co2 stored") & (c.df.marginal_cost <= -100.0)
            ]
            c.df.loc[items, "marginal_cost"] = -20.0

        marginal_costs = p * c.df.marginal_cost

        marginal_costs_grouped = marginal_costs.groupby(c.df.carrier).sum()

        marginal_costs_grouped = pd.concat([marginal_costs_grouped], keys=["marginal"])
        marginal_costs_grouped = pd.concat([marginal_costs_grouped], keys=[c.list_name])

        costs = costs.reindex(marginal_costs_grouped.index.union(costs.index))

        costs.loc[marginal_costs_grouped.index, label] = marginal_costs_grouped
    return costs

columns = pd.MultiIndex.from_tuples(
        [(27, 'vopt', '3H3H', 2045)],
        names=["cluster", "ll", "opt", "planning_horizon"],
    )

supply_energy = pd.DataFrame(columns=columns, dtype=float)

def assign_carriers(n):
    if "carrier" not in n.lines:
        n.lines["carrier"] = "AC"


def assign_locations(n):
    for c in n.iterate_components(n.one_port_components | n.branch_components):
        ifind = pd.Series(c.df.index.str.find(" ", start=4), c.df.index)
        for i in ifind.unique():
            names = ifind.index[ifind == i]
            c.df.loc[names, "location"] = "" if i == -1 else names.str[:i]

# assign_locations(n)
# assign_carriers(n)

calculate_costs(n, (27, 'vopt', '3H3H', 2045), supply_energy)

                                                                           bus0  \
Link                                                                              
relation/10377412-320-DC                                                  FR0 0   
relation/13295785-515-DC                                                  NO1 0   
relation/14126301-450-DC                                                  GB2 0   
relation/15772117-320-DC                                                  GB2 0   
relation/15781671-525-DC                                                  GB2 0   
relation/16213216-525-DC                                                  DE0 0   
relation/17631956-250-DC                                                  ES0 0   
relation/2127794-270-DC                                                   GB2 0   
relation/2505320-400-DC                                                   DK1 0   
relation/3391794-200-DC                                                   IT3 0   
rela

cluster                                                                      27
ll                                                                         vopt
opt                                                                        3H3H
planning_horizon                                                           2045
generators    capital  biogas                                            0.0000
                       coal                                              0.0000
                       gas                                               0.0000
                       geothermal heat                                   0.0000
                       nuclear                                 74158195641.9979
                       offwind-ac                               3991606617.8404
                       offwind-dc                               2958314284.2359
                       offwind-float                            1388982829.4892
                       oil primary                                       0.0000
                       onwind                                 166969445307.3546
                       ror                                      9999550786.9411
                       rural solar thermal                            2894.2045
                       solar                                    7604372371.2376
                       solar rooftop                            2702715883.0196
                       solar-hsat                              10724503137.0049
                       solid biomass                                     0.0000
                       urban central solar thermal                    3128.8411
                       urban decentral solar thermal                  3158.4471
              marginal biogas                                  17668269817.1683
                       coal                                      237777630.6000
                       gas                                      1145388101.5695
                       geothermal heat                                   0.2599
                       nuclear                                  6790347433.6519
                       offwind-ac                                  1969612.3616
                       offwind-dc                                   987281.4681
                       offwind-float                                293595.7109
                       oil primary                             15124183296.0841
                       onwind                                    139874914.2060
                       ror                                         1156901.3800
                       rural solar thermal                               0.2765
                       solar                                       2160728.7728
                       solar rooftop                               1043957.0554
                       solar-hsat                                  5645269.2700
                       solid biomass                           10107248724.3130
                       urban central solar thermal                       0.6525
                       urban decentral solar thermal                     0.3296
lines         capital  AC                                       5316765462.1147
links         capital  BEV charger                                       0.0000
                       DAC                                     29060694731.6274
                       DC                                      15857427743.9546
                       Fischer-Tropsch                         14484713077.2441
                       H2 Electrolysis                        108610791364.7393
                       H2 Fuel Cell                                   3937.1111
                       H2 pipeline                              1167178983.8157
                       OCGT                                           4626.0413
                       SMR                                            4580.8623
